# CIFAR-10

In this last example, we will be using the famous CIFAR-10 dataset. CIFAR-10 is a large image dataset containing over 60,000 images representing 10 different classes of objects like cats, planes, and cars. The images are full-color RGB, but unfortunately they are fairly small, only 32 x 32.


In the exercise below, we still use the somewhat older version of constructing a convolutional neural network, and a seperate image dataloader. In the previous notebook, we also showed the newer version, and that is runs a lot faster. 
We encourage you to also try this newer way... 

<img src="./resources/keras.png"  style="height: 350px"/>

## Step 1. Loading and preprocessing the data

The first thing we should do is import the necessary libraries.

In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

Now let's load in the dataset. We can do so simply by specifying which variables we want to load the data into, and then using the load_data() function from cifar10.

In [2]:
# loading in the data

from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

One thing we want to do is normalize the input data. If the values of the input data are in too wide a range it can negatively impact how the network performs. In this case, the input values are the pixels in the image, which have a value between 0 to 255.

So in order to normalize the data we can simply divide the image values by 255. To do this we first need to make the data a float type, since they are currently integers. We can do this by using the astype() Numpy command.

In [3]:
# normalize the inputs from 0-255 to between 0 and 1 by dividing by 255
    
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

### Integer Encoding vs. One-Hot Encoding

Now let's have a look at the values in y_train.

In [4]:
print(y_train)

[[6]
 [9]
 [9]
 ...
 [9]
 [1]
 [1]]


As you can see every category in our dataset is assigned an integer value. For example, frog is 6, truck is 9, and automobile is 1. https://keras.io/api/datasets/cifar10/


This is called *label encoding* or __integer encoding__. Integer values have a natural ordered relationship between each other and machine learning algorithms may try to make benefit of this relationship. In the case of categorical variables where no such ordinal relationship exists, this of course is complete nonsense. This would mean for example that bird (2) is less than cat (3). What should be the meaning of this?

In this case, __one-hot encoding__ is the solution. This is where the integer encoded variable is removed and a new binary variable is added for each unique integer value.

In our case, there are 10 categories and therefore 10 binary variables are needed. A __one__ value is placed in the binary variable for the corresponding category and __zero__ values for the other categories.

So: frog is [0 0 0 0 0 1 0 0 0 0] and truck is [0 0 0 0 0 0 0 0 0 1]. By the way, we used one-hot encoding as well for the cats [0 1] and dogs [1 0] classification. 

The Numpy command `to_categorical()` is used to one-hot encode the integer categories. We also need to specify the number of classes that are in the dataset, so we know how many neurons the final layer will contain.

In [5]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

class_num = y_test.shape[1]

In [6]:
print(class_num)

10


In [7]:
y_test.shape

(10000, 10)

Compare the result below with the original values of `y_train`.

In [8]:
print(y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


Let's have a quick look at the shape of one image. By now, you know what these values mean. Right?

In [9]:
print(X_train.shape[1:])

(32, 32, 3)


In [10]:
X_train.shape

(50000, 32, 32, 3)

## Step 2. Designing the Model - Exercise

We've reached the stage where we design the CNN model. I will describe the properties of the model. It's up to you to code the model using Keras.

__a) Model type__
- The first thing to do is to define the format we would like to use for the model. Keras has several different formats, but `Sequential` is the most commonly used.

__b) First convolution__
- The first layer of our model is a `convolutional` layer. The number of filters we want is `32`, the size of the filter we want is `3x3`. Don't forget to specify the `input shape` (when creating the first layer). We will use `relu`, the most common activation function. Because we don't want to change the size of the image, add `padding='same'`.

- Now make a dropout layer to prevent overfitting, which functions by randomly eliminating some of the connections between the layers. Drop 20% of the existing connections.

- Add a batch normalization layer. Batch normalization is a technique for improving the speed, performance, and stability of Neural Networks. The reasons behind its effectiveness involve a lot of math, which is beyond the scope of this course. But if you are interested, you can find a lot of information on the internet.

__c) Second convolution__
- Add a `convolutional` layer, once again. Use `64` filters this time. The size is `3x3`, activation is `relu` and `padding='same'`.
- Add a MaxPooling layer with size 2x2.
- Add a Dropout layer (20%).
- Conclude with a batch normalization layer.

__d) Third convolution__
- Repeat exactly the second convolution.

__e) Fourth convolution__
- Repeat the second convolution with the following changes: Use 128 filters. Omit the MaxPooling layer. Since the images are so small here already, we won't pool more than twice.

__f) Flatten__
- After you are done with the convolutional layers, you need to flatten the data. To prevent overfitting, add a Dropout layer once more.

__g) First dense layer__
- Add a dense layer with 256 neurons. The activation funcion is `relu`.
- Add a Dropout layer (20%).
- Conclude with a batch normalization layer.

__h) Second dense layer__
- Repeat the first dense layer, but with 128 neurons this time. Note that the number of neurons in succeeding layers decreases, eventually approaching the same number of neurons as there are classes in the dataset (in this case 10). 

__i) Final layer__
- In the final layer, we pass in the number of classes for the number of neurons. Each neuron represents a class, and the output of this layer will be a 10 neuron vector with each neuron storing some probability that the image in question belongs to the class it represents. Finally, the softmax activation function selects the neuron with the highest probability as its output, voting that the image belongs to that class (sigmoid function is used for two classes, whereas the softmax function is used for the multiclasses).

## Step 3. Compiling the model - Exercise

Now that you've designed the model, you just have to compile it. Let's specify the number of epochs we want to train for, as well as the optimizer we want to use.

The optimizer is what will tune the weights in your network to approach the point of lowest loss. The Adam algorithm is one of the most commonly used optimizers because it gives great performance on most problems. Use `categorical_crossentropy` as loss function and `accuracy` as metrics.

Print out the model summary to see what the whole model looks like. The total number of parameters should be 2,264,458.

## Step 4. Training the model - Exercise

Let's train our model now! To do this, all we have to do is call the fit() function on the model and pass in the chosen parameters. We will store the training loss values and metrics in a history object, so we can visualize the training process later.

We are going to train the model in 15 epochs, using a batch size of 64. We'll be training on 50,000 samples and validating on 10,000 samples. Since our CNN is rather complex, this will take a very long time. 

So it might be a better idea to train the model in 15 epochs at home, while doing other things. For now, train the model in 1 epoch. The accuracy will be crap, but you can at least complete the rest of the Notebook. Later at home, you can try to achieve a better accuracy.

In [ ]:
history = model.fit( ... )

__It might be a good idea to save the weights of your trained model!__

In [ ]:
model.save_weights('saved_models/modelcifar-10.h5')

## Step 5. Visualizing the training process

With this simple function we will be able to plot our training history.

In [ ]:
import matplotlib.pyplot as plt

def plotLosses(history):  
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
plotLosses(history)

## Step 6. Make predictions - Exercise

With the model trained, we can use it to make predictions about the test images.

Draw the first 25 __test images__ (5 images in a row). Below the image you print the actual value (a) and the predicted value (p). If they are the same the textcolor is green, red otherwise. Tune the subplot layout and create a little bit of space between the subplots. You should get something like this:

<img src="./resources/uit.png"/>